In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def preprocess_data():
    data = {
    'id': [1, 2, 3, 4, 5],
    'title': ['Banwait food court','RoadSide A Food Joint','Courtyard Pavilion','Maharashtra Lunch Home ','Giza Food Corner'],
    'price': [1500, 800, 2000, 3000, 1000],
    'lat': [19.033696, 19.024384, 19.043365, 19.042841, 19.031753],
    'lng': [73.016750, 73.019967, 73.027075, 73.021976, 73.021614],
    'capacity': [10, 20, 50, 5, 30],
    'rating': ['4.5', '4.0', '3.8', '4.9', '3.5'],
    'amenities': ['Area','street','AC','regional','fastfood'],
            }
    df = pd.DataFrame(data)
     
    df['rating'] = df['rating'].astype(float)

    # Standardize amenities names
    amenities_mapping = {
        'Area': 'Indoor Seating Area',
        'street': 'Street Parking',
        'AC': 'Air Conditioning',
        'regional': 'Regional Cuisine',
        'fastfood': 'Fast Food'
    }
    df['amenities'] = df['amenities'].replace(amenities_mapping)

    # Convert list-like amenities to separate rows (if applicable)
    if df['amenities'].apply(lambda x: isinstance(x, list)).any():
        df = df.explode('amenities')

    # One-hot encode amenities
    amenities_dummies = df['amenities'].str.get_dummies(sep=',')
    df = pd.concat([df, amenities_dummies], axis=1)

    # Normalize price and capacity
    df[['price', 'capacity']] = df[['price', 'capacity']].fillna(df[['price', 'capacity']].median())
    scaler = MinMaxScaler()
    df[['price_norm', 'capacity_norm']] = scaler.fit_transform(df[['price', 'capacity']])
    
    df.to_csv("processed_spaces.csv", index=False)
    return df

if __name__ == "__main__":
    preprocess_data()


,id,title,price,lat,lng,capacity,rating,amenities
0,1,Banwait food court,1500,19.033696,73.016750,10,4.5,Indoor Seating Area
1,2,RoadSide A Food Joint,800,19.024384,73.019967,20,4.0,Street Parking
2,3,Courtyard Pavilion,2000,19.043365,73.027075,50,3.8,Air Conditioning
3,4,Maharashtra Lunch Home,3000,19.042841,73.021976,5,4.9,Regional Cuisine
4,5,Giza Food Corner,1000,19.031753,73.021614,30,3.5,Fast Food


In [19]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import joblib

def train_recommendation_model():
    df = pd.read_csv(r"D:\mini_project1\miniproject1\data\processed_spaces.csv")
    features = df[['price_norm', 'capacity_norm', 'Air Conditioning', 'Fast Food']]  # Add all amenity columns
    
    # Train KNN
    model = NearestNeighbors(n_neighbors=3, metric='cosine')
    model.fit(features)
    joblib.dump(model, r"D:\mini_project1\miniproject1\models\space_recommender.pkl")

if __name__ == "__main__":
    train_recommendation_model()

In [21]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def preprocess_data():
    data = {
    'id': [1, 2, 3, 4, 5],
    'title': ['Banwait food court','RoadSide A Food Joint','Courtyard Pavilion','Maharashtra Lunch Home ','Giza Food Corner'],
    'price': [1500, 800, 2000, 3000, 1000],
    'lat': [19.033696, 19.024384, 19.043365, 19.042841, 19.031753],
    'lng': [73.016750, 73.019967, 73.027075, 73.021976, 73.021614],
    'capacity': [10, 20, 50, 5, 30],
    'rating': ['4.5', '4.0', '3.8', '4.9', '3.5'],
    'amenities': ['Area','street','AC','regional','fastfood'],
            }
    df = pd.DataFrame(data)
     
    df['rating'] = df['rating'].astype(float)

    # Standardize amenities names
    amenities_mapping = {
        'Area': 'Indoor Seating Area',
        'street': 'Street Parking',
        'AC': 'Air Conditioning',
        'regional': 'Regional Cuisine',
        'fastfood': 'Fast Food'
    }
    df['amenities'] = df['amenities'].replace(amenities_mapping)

    # Convert list-like amenities to separate rows (if applicable)
    if df['amenities'].apply(lambda x: isinstance(x, list)).any():
        df = df.explode('amenities')

    # One-hot encode amenities
    amenities_dummies = df['amenities'].str.get_dummies(sep=',')
    df = pd.concat([df, amenities_dummies], axis=1)

    # Normalize price and capacity
    df[['price', 'capacity']] = df[['price', 'capacity']].fillna(df[['price', 'capacity']].median())
    scaler = MinMaxScaler()
    df[['price_norm', 'capacity_norm']] = scaler.fit_transform(df[['price', 'capacity']])
    
    df.to_csv(r"D:\mini_project1\miniproject1\data\processed_spaces.csv", index=False)
    return df

if __name__ == "__main__":
    preprocess_data()


In [22]:
from flask import Flask, request, jsonify
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler

app = Flask(__name__)
model = joblib.load(r"D:\mini_project1\miniproject1\models\space_recommender.pkl")
df = pd.read_csv(r"D:\mini_project1\miniproject1\data\processed_spaces.csv")

@app.route('/recommend', methods=['POST'])
def recommend():
    user_input = request.json  # e.g., {"price": 2000, "capacity": 15, "amenities": ["AC"]}
    # Preprocess input
    input_data = [[
        user_input['price'] / 3000,  # Manual normalization (adjust max price)
        user_input['capacity'] / 50,  # Manual normalization (adjust max capacity)
        1 if "AC" in user_input['amenities'] else 0,
        1 if "Parking" in user_input['amenities'] else 0,
        1 if "Indoor" in user_input['amenities'] else 0
    ]]
    # Get recommendations
    distances, indices = model.kneighbors(input_data)
    recommendations = df.iloc[indices[0]].to_dict('records')
    return jsonify(recommendations)

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\balar\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
